In [ ]:
"""
stereo_calibrate_high_accuracy.py
High-accuracy stereo calibration using findChessboardCornersSB (recommended).
Adjust user settings below and run.

Outputs:
 - stereo_params.npz (mtxL/distL/mtxR/distR/R/T/E/F/R1/R2/P1/P2/Q/map1x/map1y/map2x/map2y)
 - stereo_params_debug.npz (includes filtered pair indices and per-image errors)
 - optional rectified_check.png for visual inspection
"""

import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import json
from datetime import datetime

# ---------------- USER SETTINGS ----------------
left_dir = r"C:\Users\ASUS\Downloads\Semester 5_Automotive\Computer Vision\FINAL\VeryVeryNewVehicleDetection\car and truck detection.v3i.yolov8\Fix\stereo_pairs\L"
right_dir = r"C:\Users\ASUS\Downloads\Semester 5_Automotive\Computer Vision\FINAL\VeryVeryNewVehicleDetection\car and truck detection.v3i.yolov8\Fix\stereo_pairs\R"

pattern = (13, 9)           # inner corners (nx, ny) from your PDF (13x9)
square_size = 0.02          # meters (20 mm)
keep_best_ratio = 0.9       # keep best 90% pairs by mean reprojection error
min_pairs_required = 8
visualize = True            # show matplotlib plots during run (set False for headless)
save_rectified_preview = True
failed_dir = "failed_pairs_sb"
# ------------------------------------------------

# helper functions
def gather_imgs(folder):
    imgs = []
    for ext in ("*.png","*.jpg","*.jpeg","*.bmp","*.tif"):
        imgs += glob.glob(os.path.join(folder, ext))
    return sorted(imgs)

def extract_index(path):
    # extract first integer group from filename
    import re
    m = re.search(r"(\d+)", os.path.basename(path))
    return int(m.group(1)) if m else None

def ensure_consistent_order(corners):
    """
    Basic heuristic to ensure consistent ordering: if first corner x > last corner x, flip.
    Accepts Nx1x2 array; returns Nx1x2 np.float32
    """
    pts = corners.reshape(-1,2)
    if pts[0,0] > pts[-1,0]:
        pts = pts[::-1]
    return pts.reshape(-1,1,2).astype(np.float32)

# ------------------------------------------------------------------
# 1) gather images and pair by numeric index
# ------------------------------------------------------------------
left_imgs = gather_imgs(left_dir)
right_imgs = gather_imgs(right_dir)

if not left_imgs or not right_imgs:
    raise SystemExit("No images found in one of the directories. Check paths.")

Lmap = {extract_index(p): p for p in left_imgs}
Rmap = {extract_index(p): p for p in right_imgs}
common_ids = sorted(set(Lmap.keys()) & set(Rmap.keys()))
pairs = [(Lmap[i], Rmap[i]) for i in common_ids]

if len(pairs) == 0:
    raise SystemExit("No paired images found by numeric index. Ensure filenames contain matching indices.")

print(f"Found {len(left_imgs)} left, {len(right_imgs)} right, paired: {len(pairs)}")

os.makedirs(failed_dir + "/L", exist_ok=True)
os.makedirs(failed_dir + "/R", exist_ok=True)

# ------------------------------------------------------------------
# 2) prepare object points
# ------------------------------------------------------------------
objp_unit = np.zeros((pattern[0]*pattern[1], 3), np.float32)
objp_unit[:,:2] = np.mgrid[0:pattern[0], 0:pattern[1]].T.reshape(-1,2)
objp_unit *= square_size

objpoints = []
imgpointsL = []
imgpointsR = []
pair_paths = []

# ------------------------------------------------------------------
# 3) detect corners using SB (robust)
# ------------------------------------------------------------------
use_sb = hasattr(cv2, "findChessboardCornersSB")
if not use_sb:
    print("Warning: findChessboardCornersSB not available in this OpenCV build. The script expects SB for best results.")

criteria_subpix = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 40, 1e-4)

print("\nDetecting corners (this may take a bit)...")
for idx, (Lp, Rp) in enumerate(pairs):
    Il = cv2.imread(Lp)
    Ir = cv2.imread(Rp)
    if Il is None or Ir is None:
        print(f"Skipping pair {idx} - could not read files")
        continue
    gl = cv2.cvtColor(Il, cv2.COLOR_BGR2GRAY)
    gr = cv2.cvtColor(Ir, cv2.COLOR_BGR2GRAY)
    if use_sb:
        foundL, cornersL = cv2.findChessboardCornersSB(gl, pattern, flags=cv2.CALIB_CB_NORMALIZE_IMAGE)
        foundR, cornersR = cv2.findChessboardCornersSB(gr, pattern, flags=cv2.CALIB_CB_NORMALIZE_IMAGE)
    else:
        foundL, cornersL = cv2.findChessboardCorners(gl, pattern, flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE)
        foundR, cornersR = cv2.findChessboardCorners(gr, pattern, flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE)

    print(f"[{idx:03d}] L={bool(foundL)} R={bool(foundR)}")

    if foundL and foundR:
        # cornerSubPix
        cornersL = cv2.cornerSubPix(gl, cornersL, (11,11), (-1,-1), criteria_subpix)
        cornersR = cv2.cornerSubPix(gr, cornersR, (11,11), (-1,-1), criteria_subpix)

        # ensure consistent ordering
        cornersL = ensure_consistent_order(cornersL)
        cornersR = ensure_consistent_order(cornersR)

        objpoints.append(objp_unit.copy())
        imgpointsL.append(cornersL.astype(np.float32))
        imgpointsR.append(cornersR.astype(np.float32))
        pair_paths.append((Lp, Rp))
    else:
        # save bad visual for debugging
        cv2.imwrite(os.path.join(failed_dir, "L", f"failed_{idx:03d}.png"), Il)
        cv2.imwrite(os.path.join(failed_dir, "R", f"failed_{idx:03d}.png"), Ir)

print(f"\nValid stereo detections: {len(objpoints)}")

if len(objpoints) < min_pairs_required:
    raise SystemExit(f"Too few valid pairs ({len(objpoints)}) — collect more images (>= {min_pairs_required}).")

# image shape for calibrations
img_shape = cv2.cvtColor(cv2.imread(pair_paths[0][0]), cv2.COLOR_BGR2GRAY).shape[::-1]

# ------------------------------------------------------------------
# 4) mono calibrations
# ------------------------------------------------------------------
print("\nCalibrating individual cameras (mono)...")
flags_mono = 0  # allow full distortion estimation
retL, K1, dist1, rvecsL, tvecsL = cv2.calibrateCamera(objpoints, imgpointsL, img_shape, None, None, flags=flags_mono)
retR, K2, dist2, rvecsR, tvecsR = cv2.calibrateCamera(objpoints, imgpointsR, img_shape, None, None, flags=flags_mono)

print(f"Left RMS: {retL:.4f}, Right RMS: {retR:.4f}")

# compute per-image reprojection errors
def per_image_errors(objpts, imgpts, rvecs, tvecs, K, dist):
    errs = []
    for o,i,rv,tv in zip(objpts, imgpts, rvecs, tvecs):
        proj,_ = cv2.projectPoints(o, rv, tv, K, dist)
        proj = proj.reshape(-1,2)
        i2 = i.reshape(-1,2)
        errs.append(np.mean(np.linalg.norm(proj - i2, axis=1)))
    return np.array(errs)

errsL = per_image_errors(objpoints, imgpointsL, rvecsL, tvecsL, K1, dist1)
errsR = per_image_errors(objpoints, imgpointsR, rvecsR, tvecsR, K2, dist2)
mean_errs = (errsL + errsR) / 2.0

# plot per-image errors
if visualize:
    plt.figure()
    plt.title("Per-image mean reprojection error (px)")
    plt.plot(mean_errs, "o-")
    plt.xlabel("Pair index")
    plt.ylabel("Mean reproj error (px)")
    plt.grid(True)
    plt.show()

# ------------------------------------------------------------------
# 5) keep best pairs (filter out bad frames)
# ------------------------------------------------------------------
n_keep = max(min_pairs_required, int(len(mean_errs) * keep_best_ratio))
keep_idx = np.argsort(mean_errs)[:n_keep]
print(f"Keeping {n_keep} / {len(mean_errs)} pairs (best by reprojection error)")

objpoints_f = [objpoints[i] for i in keep_idx]
imgpointsL_f = [imgpointsL[i] for i in keep_idx]
imgpointsR_f = [imgpointsR[i] for i in keep_idx]
pair_paths_f = [pair_paths[i] for i in keep_idx]

# Optionally re-run mono calibrations on filtered data to refine (recommended)
print("\nRefining mono calibration on filtered pairs...")
retL_f, K1_f, dist1_f, rvecsL_f, tvecsL_f = cv2.calibrateCamera(objpoints_f, imgpointsL_f, img_shape, K1, dist1, flags=0)
retR_f, K2_f, dist2_f, rvecsR_f, tvecsR_f = cv2.calibrateCamera(objpoints_f, imgpointsR_f, img_shape, K2, dist2, flags=0)
print(f"Refined Left RMS: {retL_f:.4f}, Right RMS: {retR_f:.4f}")

# ------------------------------------------------------------------
# 6) stereo calibrate (fix intrinsics for stability)
# ------------------------------------------------------------------
print("\nRunning stereoCalibrate (CALIB_FIX_INTRINSIC)...")
stereo_flags = cv2.CALIB_FIX_INTRINSIC
stereo_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, 1e-6)

retStereo, _, _, _, _, R, T, E, F = cv2.stereoCalibrate(
    objpoints_f, imgpointsL_f, imgpointsR_f,
    K1_f, dist1_f, K2_f, dist2_f,
    img_shape,
    criteria=stereo_criteria,
    flags=stereo_flags
)

print(f"Stereo RMS: {retStereo:.6f}")
print("R:\n", R)
print("T:\n", T.ravel())
baseline = np.linalg.norm(T)  # in meters because objp used meters
print(f"Baseline (m): {baseline:.4f}  (mm: {baseline*1000:.1f})")

# Optional single safe refinement (uncomment with caution)
# print("\nOptional safe refinement (UNCOMMENT to enable)")
# refine_flags = cv2.CALIB_USE_INTRINSIC_GUESS
# retStereo_ref, _, _, _, _, R_ref, T_ref, E_ref, F_ref = cv2.stereoCalibrate(
#     objpoints_f, imgpointsL_f, imgpointsR_f,
#     K1_f, dist1_f, K2_f, dist2_f,
#     img_shape, criteria=stereo_criteria, flags=refine_flags
# )
# print("Refined stereo RMS:", retStereo_ref)

# ------------------------------------------------------------------
# 7) stereoRectify + initUndistortRectifyMap
# ------------------------------------------------------------------
print("\nComputing stereoRectify and maps...")
R1, R2, P1, P2, Q, valid1, valid2 = cv2.stereoRectify(
    K1_f, dist1_f, K2_f, dist2_f, img_shape, R, T, flags=cv2.CALIB_ZERO_DISPARITY, alpha=0
)

map1x, map1y = cv2.initUndistortRectifyMap(K1_f, dist1_f, R1, P1, img_shape, cv2.CV_16SC2)
map2x, map2y = cv2.initUndistortRectifyMap(K2_f, dist2_f, R2, P2, img_shape, cv2.CV_16SC2)

# ------------------------------------------------------------------
# 8) save results
# ------------------------------------------------------------------
out_npz = "stereo_params.npz"
np.savez(out_npz,
         K1=K1_f, dist1=dist1_f, K2=K2_f, dist2=dist2_f,
         R=R, T=T, E=E, F=F,
         R1=R1, R2=R2, P1=P1, P2=P2, Q=Q,
         map1x=map1x, map1y=map1y, map2x=map2x, map2y=map2y)

# debug/extra info
debug_npz = "stereo_params_debug.npz"
np.savez(debug_npz,
         keep_idx=keep_idx, mean_errs=mean_errs,
         objpoints_f=objpoints_f, imgpointsL_f=imgpointsL_f, imgpointsR_f=imgpointsR_f,
         K1=K1_f, dist1=dist1_f, K2=K2_f, dist2=dist2_f, R=R, T=T)

print(f"\nSaved {out_npz} and {debug_npz}")

# also write a JSON summary for quick reading
summary = {
    "timestamp": datetime.now().isoformat(),
    "num_pairs_total": len(pairs),
    "num_pairs_used": len(objpoints_f),
    "mono_left_rms": float(retL_f),
    "mono_right_rms": float(retR_f),
    "stereo_rms": float(retStereo),
    "baseline_m": float(baseline),
    "K1": K1_f.tolist(),
    "K2": K2_f.tolist()
}
with open("stereo_params_summary.json","w") as f:
    json.dump(summary, f, indent=2)

# ------------------------------------------------------------------
# 9) Visual checks (rectified preview and error plots)
# ------------------------------------------------------------------
if save_rectified_preview:
    try:
        i0 = int(keep_idx[0])
        imL = cv2.imread(pair_paths[i0][0])
        imR = cv2.imread(pair_paths[i0][1])
        rL = cv2.remap(imL, map1x, map1y, interpolation=cv2.INTER_LINEAR)
        rR = cv2.remap(imR, map2x, map2y, interpolation=cv2.INTER_LINEAR)
        concat = np.hstack((rL, rR))
        # draw horizontal guide lines
        h = concat.shape[0]
        for y in range(50, h, 50):
            cv2.line(concat, (0,y), (concat.shape[1], y), (0,0,0), 1)
        cv2.imwrite("rectified_check.png", concat)
        print("Saved rectified preview: rectified_check.png")
    except Exception as e:
        print("Could not generate rectified preview:", e)

if visualize:
    # histogram of mean errors
    plt.figure()
    plt.title("Histogram of mean per-pair reprojection errors (px)")
    plt.hist(mean_errs, bins=20)
    plt.xlabel("mean reproj error (px)")
    plt.show()

print("\nCalibration finished. Inspect stereo_params_summary.json and rectified_check.png")


ini code yang stereo calibration